In [1]:
from tc import config

In [2]:
import tc.config as config

In [12]:
#main.py file

import numpy as np
import pandas as pd
import copy
import tc.config as config
# from tc.data_import import ImportData
from tc.distance_calc import DistanceCalculation
from tc.results_page import ResultCalculations ,calculate_lift

In [141]:
# Segregating Columns

categorical_columns = ['Zip','ID','TC']
numerical_columns = ['NRx','PDE','Copay',
                     'Delivered_Doximity','Delivered_Epocrates','Engaged_Doximity',
                     'Engaged_Epocrates','Delivered_Medscape','Engaged_Medscape',
                     'Delivered_Nexgen','Engaged_Nexgen','SpeakerEvent']
hcp_identifier = 'ID'


date_columns='Month'
date_format='yyyymm'
format = ['long']
tc_identifier='TC'
event_identifer='SpeakerEvent'
test_value='T'
control_value='C'
segment_var=[]


format_mapping = {
    "yyyy-mm": "%Y-%m",
    "yyyymm": "%Y%m",
    "dd-mm-yyyy": "%d-%m-%Y",
    "mm-yyyy": "%m-%Y",
    "yyyy-mm-dd": "%Y-%m-%d",
    
}




pre_start='Jul-2019'
pre_end='Dec-2019'  
event_date='Jan-2020'
post_end='Jun-2020'

# For aggregration variable specification
#cases = {'Emails': [5],'Calls':[5],'Brand_Sales':[5]}

# Define cases specifying positions for each prefix
cases = {
    'NRx': {6: ['pre']},   
    #'Mobile_Alerts': {8: ['pre']}
                  # For prefix 'B', select columns from the end
           # For prefix 'B', select columns from the beginning
}

event_vars =['NRx']
agg_vars=['NRx']
sales_var='NRx'


mom_pre_wts=[1,1,1]
mom_post_wts=[1,1,1]

agg_pre_wts=[1,1,1]
agg_post_wts=[1,1,1]

mom_pre_match=True
mom_post_match=False

agg_pre_match=True
agg_post_match=False


#Defining batch size

batch_size = 50
result_df = pd.DataFrame(columns=['Test_ID', 'Control_ID', 'Euclidean Distance'])

In [134]:
import pandas as pd
from tomlkit import date
import tc.config as config
import pdb
from tc.log_config import setup_logging
setup_logging()

import logging
# Import Module:

class ImportData:
    def __init__(
        self,
        file_path: str,
        data_format: str,
        date_format: str,
        date_col: str,
        categorical_cols: list,
        numerical_cols: list,
        identifier: str,
        week : bool
    ):
        self.file_path = file_path
        self.data_format = data_format
        self.date_format = date_format
        self.date_col = date_col
        self.categorical_cols = categorical_cols
        self.numerical_cols = numerical_cols
        self.identifier = identifier
        self.week=week

    def read_csv(self):
        try:
            df = pd.read_csv(self.file_path)
        except FileNotFoundError:
            logging.error(f"File not found at {self.file_path}")
            raise FileNotFoundError(f"File not found at {self.file_path}")
        except Exception as e:
            logging.error(f"Unexpected error occurred while reading {self.file_path} - {str(e)}")
            raise Exception(f"Unexpected error occurred: {str(e)}")
        else:
            if self.data_format == "long":
                df = self.pivot_long_to_wide(df)

            logging.info(f"File {self.file_path} imported successfully.")

            return df

    def date_conversion(self, data: pd.DataFrame) -> pd.DataFrame:
        try:
            data = data.sort_values(self.date_col)
            if self.date_format in format_mapping:
                format_string = format_mapping[self.date_format]
                data[self.date_col] = pd.to_datetime(
                    data[self.date_col], format=format_string
                ).dt.strftime("%b%y")
            logging.info(f"Date conversion was successfully.")
            return data
        except ValueError as e:
            logging.error(f"Date conversion error: {e}.")
            raise Exception(f"Error: {e}. Please provide correct date format")


    def pivot_long_to_wide(self, data) -> pd.DataFrame:
        # try:
        # Pivot the dataframe
        data = data.filter(
            regex=f"^({self.numerical_cols}|{self.categorical_cols}|{self.date_col})",
            axis=1
        )
        data = self.date_conversion(data=data)
        unique_date_values = list(data[self.date_col].unique())
        data["Date1"] = pd.Categorical(
            data[self.date_col], categories=unique_date_values, ordered=True
        )
        print('Here1')
        pivot_df = data.pivot_table(
            index=self.identifier, columns="Date1", values=self.numerical_cols # type: ignore
        )
        print('Here1')

        # Flatten the multi-level column index
        pivot_df.columns = [
            "_".join(col).rstrip("_") for col in pivot_df.columns.values
        ]
        # Reset the index
        pivot_df = pd.DataFrame(pivot_df)
        pivot_df.reset_index(inplace=True)
        merged_data = pd.merge(
            pivot_df,
            data.drop_duplicates(subset=[self.identifier]),
            on=self.identifier,
        )
        logging.info(f"Pivoting was successfully.")
        return merged_data
        # except Exception as e:
        #     logging.error(f"An error occurred during pivoting: {e}")
        #     print(f"An error occurred:Please upload long format data,Check data for long-wide conversion")
        #     raise Exception("Pivoting error, check data format and structure.")
        #     return pd.DataFrame()

In [119]:
from logging import config
import pandas as pd
import numpy as np
import re
from tc.analysis_months_calculation import analyis_initial_months
import tc.config as config
from tc.table_creation import generate_variables
from tc.multipliers import apply_multipliers_table
from tc.time_align_var_func import increment_numbers_in_list
from tc.agg_logic import calculate_sum_last_n_columns,standardize_columns
from scipy.spatial.distance import cdist
import pdb
from itertools import product

In [145]:
pre_start_var='Jul-2019'
pre_end_var='Dec-2019'  
event_date_var='Jan-2020'
post_end_var='Jun-2020'

class DistanceCalculation:
    def __init__(self,data,num_cols,hcp_identifier,tc_identifier,event_identifier):
        self.data = data
        self.num_cols=num_cols
        self.date_length=None
        self.hcp_identifier=hcp_identifier
        self.tc_identifier=tc_identifier
        self.event_identifier=event_identifier

    def time_align(self,test_filter_value,post_cut_off)-> pd.DataFrame:
        selected_data = self.data.filter(regex=f'^({self.hcp_identifier}|{self.event_identifier}|{self.tc_identifier})', axis=1)
        selected_data=selected_data[selected_data[self.tc_identifier] == test_filter_value]
        selected_columns=list(selected_data)
        selected_columns.remove(self.hcp_identifier)
        selected_columns.remove(self.tc_identifier)
        num_months=len(selected_columns)
        self.date_length=num_months
        mapping_dict={'Months':selected_columns,'Index': list(range(1,num_months+1))}
        selected_data.drop([self.tc_identifier],inplace=True,axis=1)
        pre_post_calc=analyis_initial_months(event_date=event_date_var,pre_start=pre_start_var,post_end= post_end_var,pre_end=pre_end_var)
        self.mapping_month=mapping_dict['Index'][mapping_dict['Months'].index(f"{self.event_identifier}_{pre_post_calc['event_date']}")]
        first_exposure=selected_data.loc[:,  f"{self.event_identifier}_{pre_post_calc['event_date']}":].gt(0).idxmax(axis=1).apply(lambda x:str(x))
        # Create a mapping dictionary using zip
        mapping_dict = dict(zip(mapping_dict['Months'], mapping_dict['Index']))
        # Replace values in the "Post_Start" column based on the mapping dictionary
        time_align=pd.DataFrame({self.hcp_identifier: selected_data[self.hcp_identifier], 'Post_Start': first_exposure})
        time_align['Post_Start'] = time_align['Post_Start'].replace(mapping_dict)
        time_align['Post_End']=time_align['Post_Start']+pre_post_calc['total_months_post']-1
        time_align['Pre_End']=time_align['Post_Start']-1
        time_align['Pre_Start']=time_align['Post_Start']-pre_post_calc['total_months_pre']
        #condition for boundary cases
        time_align['Post_End'] = time_align['Post_End'].apply(lambda x: x if x <= num_months else num_months)
        
        # 3 months post period cut off condition
        post_start_check = sorted(time_align['Post_Start'].unique())
        post_end_check = sorted(time_align['Post_End'].unique())
        result = []
        for i in range(len(post_start_check)):
            if i < len(post_end_check) and abs(post_end_check[i] - post_start_check[i]) <= post_cut_off:
                result.append(post_start_check[i])
        if len(result)>=0:
             time_align=time_align[~time_align['Post_Start'].isin(result)]
        self.time_aligned_data=time_align
        return time_align
    

    def column_conversion(self):
        date_length=self.date_length
        categ_values = [f"{col}_{i}" for col in self.num_cols for i in range(1, date_length + 1)]

        # Define a pattern using regular expression
        pattern = re.compile(r'Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec',re.IGNORECASE)

        # Filter columns based on the pattern
        selected_columns = list(filter(lambda col: pattern.search(col), self.data.columns))

        for vars in segment_var:
            if vars in selected_columns:
                selected_columns.remove(vars)
                
        remaining_columns = [col for col in self.data.columns if col not in selected_columns]

        # Rename columns
        # Define the desired order of columns
        desired_colums=remaining_columns+selected_columns
        # Reorder the columns
        data = self.data.reindex(columns=desired_colums)

        new_column_names = remaining_columns + categ_values
        data.rename(columns=dict(zip(data.columns, new_column_names)), inplace=True)
        self.data=data
        self.agg_data=data.copy()
        return data

    def data_standardize(self):
    # Assuming 'data' is your DataFrame with both numerical and non-numerical columns
    # Select only the numerical columns
        numerical_columns = self.data.select_dtypes(include=['float64', 'int64']).columns
        numerical_columns=numerical_columns.delete(numerical_columns.get_loc(self.hcp_identifier))
    # Create a StandardScaler object
        #scaler = StandardScaler()
        # Fit the scaler on the selected numerical columns and transform the data
        #self.data[numerical_columns] = scaler.fit_transform(self.data[numerical_columns])
        #self.data[numerical_columns] = preprocessing.scale(self.data[numerical_columns])
         # Calculate mean and standard deviation with ddof=1
        column_means = self.data[numerical_columns].mean()
        column_stds = self.data[numerical_columns].std(ddof=1)

        # Standardize the data using mean and standard deviation with ddof=1
        standardized_data = (self.data[numerical_columns] - column_means) / column_stds

        # Update the original DataFrame with standardized values
        self.data[numerical_columns] = standardized_data
        return self.data
    
    
    

    def distance_matching(self,control_matches,agg_matching,segment_matching,filter_test_val,filter_control_val,segment_var,batch_size):
        if segment_matching==True:
            self.data['segment_var'] = self.data[segment_var].astype(str).apply(lambda x: "_".join(x), axis=1)
            self.agg_data['segment_var'] = self.agg_data[segment_var].astype(str).apply(lambda x: "_".join(x), axis=1)
        else:
            self.data['segment_var']="National"
            self.agg_data['segment_var']="National"
        uni_combo=sorted(self.time_aligned_data['Pre_Start'].unique())
        pre_post_calc=analyis_initial_months(event_date=event_date_var,pre_start=pre_start_var,post_end= post_end_var,pre_end=pre_end_var)
        mom_variables_to_match,table = generate_variables(self.time_aligned_data,event_vars,pre_wts=mom_pre_wts,post_wts=mom_post_wts,pre_match=mom_pre_match,post_match=mom_post_match)
        mom_variables_to_match.append(self.hcp_identifier)
        mom_match=increment_numbers_in_list(mom_variables_to_match,times=self.time_aligned_data['Pre_Start'].nunique()-1)
        # Create a mapping of Pre_Start to index
        pre_start_index_mapping = {value: index for index, value in enumerate(uni_combo)}
        uni_combo1 = [(pre_start_var, segment, pre_start_index_mapping[pre_start_var])
                for pre_start_var, segment in product(uni_combo, self.data['segment_var'].unique())]
        
        test_df=self.data[self.data[self.tc_identifier] == filter_test_val]
        control_df=self.data[self.data[self.tc_identifier] == filter_control_val]
        test_df=pd.merge(test_df,self.time_aligned_data,on=self.hcp_identifier)
        result_df = pd.DataFrame(columns=['Test_ID', 'Control_ID', 'Euclidean Distance']) 


        for pre_start_var,segment,index in uni_combo1:
            test_df_segmented = test_df[(test_df['Pre_Start'] == pre_start_var) & (test_df['segment_var'] == segment)]
            for start_idx in range(0, test_df_segmented.shape[0], batch_size):
                end_idx = min(start_idx + batch_size, test_df_segmented.shape[0])
                test_df_batch = test_df_segmented.iloc[start_idx:end_idx]
                test_df1=test_df_batch[(test_df_batch['Pre_Start'] == pre_start_var) & (test_df_batch['segment_var'] == segment)]
                mom_vec_filter=[x for x in mom_match[index] if x in self.data.columns]
                test_df1=pd.DataFrame(test_df1[mom_vec_filter])
                control_df1=control_df[control_df['segment_var'] == segment]
                control_df1=pd.DataFrame(control_df1[mom_vec_filter])
                apply_multipliers_table(self.time_aligned_data,test_df1, table,pre_post_calc)
                apply_multipliers_table(self.time_aligned_data,control_df1, table,pre_post_calc)
                if agg_matching==True:
                    agg_variables_to_match,table1 = generate_variables(self.time_aligned_data,agg_vars,pre_wts=agg_pre_wts,post_wts=agg_post_wts,pre_match=agg_pre_match,post_match=agg_post_match)
                    agg_variables_to_match.append(self.hcp_identifier)
                    agg_match=increment_numbers_in_list(agg_variables_to_match,times=self.time_aligned_data['Pre_Start'].nunique()-1)
                    test_df_agg=pd.merge(self.agg_data,self.time_aligned_data,on=self.hcp_identifier)
                    agg_vec_filter=[x for x in agg_match[index] if x in self.data.columns]
                    test_df_agg=test_df_agg[test_df_agg['Pre_Start'] == pre_start_var]
                    test_df_agg1=calculate_sum_last_n_columns(test_df_agg,pre_post_calc,cases,agg_vec_filter,self.mapping_month)
                    control_df_agg1=calculate_sum_last_n_columns(self.agg_data[self.agg_data[self.tc_identifier] == filter_control_val],pre_post_calc,cases,agg_vec_filter,self.mapping_month)
                    agg_data=pd.concat([test_df_agg1,control_df_agg1])
                    subset_columns_agg = [col for col in agg_data.columns if col.startswith("Sum_")]
                    standardize_columns(agg_data,subset_columns_agg)
                    agg_data=agg_data[agg_data['segment_var'] == segment]
                    apply_multipliers_table(self.time_aligned_data,agg_data,table1,pre_post_calc)
                    test_df1=pd.merge(test_df1,agg_data,on=self.hcp_identifier)
                    control_df1=pd.merge(control_df1,agg_data,on=self.hcp_identifier)
                    #test_df1 = test_df1.iloc[start_idx:end_idx]
                    new_list=mom_vec_filter
                    new_list+=subset_columns_agg
                else:
                    new_list=mom_vec_filter
                new_list = [x for x in new_list if x != self.hcp_identifier]
                distances = cdist(test_df1[new_list].values, control_df1[new_list].values)
                batch_top_n_indices = np.argsort(distances, axis=1)[:, :control_matches]
                batch_top_n_control_ids = control_df1[self.hcp_identifier].values[batch_top_n_indices]
                batch_top_n_distances = np.take_along_axis(distances, batch_top_n_indices, axis=1)
                if control_df1.shape[0]!=0:
                    col_result_df = pd.DataFrame({'Test_ID': np.repeat(test_df1[self.hcp_identifier], control_matches),
                                  'Control_ID': batch_top_n_control_ids.flatten(),
                                  'Euclidean Distance': batch_top_n_distances.flatten()})
                    result_df = pd.concat([result_df, col_result_df], ignore_index=True)
            
                
        return result_df

In [121]:
# 
# 


In [122]:
# ID,Zip,NRx,TRx,MktNRx,MktTRx,PDE,Copay,Delivered_Doximity,
# Delivered_Epocrates,Engaged_Doximity,Engaged_Epocrates,Delivered_Medscape,
# Engaged_Medscape,Delivered_Nexgen,Engaged_Nexgen,SpeakerEvent,Month,Cluster,
# TV_GRPs,Website_Sessions,Display_Impressions,Display_Clicks,Print_GRPs,OTT_IMPRS,
# TV2_GRPs,Website2_Sessions,Display2_Impressions,Display2_Clicks,Print2_GRPs,OTT2_IMPRS

In [123]:
# data['Month'].head(1)

In [124]:
# # find the datatype of the column
# data['Month'].dtypes


In [125]:
# data = pd.read_csv("MMX_Data_PDE_Reduced.csv")

In [126]:
# # convert it to str
# data['Month'] = data['Month'].astype(str)

In [127]:
# data.Month.unique()

In [105]:
data = pd.read_csv("MMX_Data_PDE_Reduced_1.csv")
data['TC'] = np.where(np.random.rand(len(data)) < 0.15, 'T', 'C')
# save the dataset
# data.to_csv("MMX_Data_PDE_Reduced.csv", index=False)
data.to_csv("MMX_Data_PDE_Reduced.csv", index=False)

In [107]:
data.Month.unique()

array(['2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06'],
      dtype=object)

In [106]:
data = pd.read_csv("MMX_Data_PDE_Reduced.csv")
data['TC'].head(1)

0    C
Name: TC, dtype: object

In [ ]:
# 

In [ ]:
# 

In [ ]:
# 

In [ ]:
# 

In [73]:
# data = pd.read_csv("MMX_Data_PDE_Reduced_1.csv")
# data.head()

,ID,Zip,NRx,TRx,MktNRx,MktTRx,PDE,Copay,Delivered_Doximity,Delivered_Epocrates,...,Display_Impressions,Display_Clicks,Print_GRPs,OTT_IMPRS,TV2_GRPs,Website2_Sessions,Display2_Impressions,Display2_Clicks,Print2_GRPs,OTT2_IMPRS
0,14934,48307,4,0,1,1,7.7,4,0,0,...,377,80,973,708,438,979,622,347,729,408
1,14934,48307,2,0,0,0,1.9,5,1,1,...,427,122,583,66,805,305,906,683,846,410
2,14934,48307,1,1,2,2,10.6,4,0,0,...,588,382,550,188,495,263,212,131,471,498
3,14934,48307,0,0,0,0,8.6,2,3,0,...,609,534,149,732,465,402,344,119,100,917
4,14934,48307,3,0,1,1,4.7,3,0,0,...,683,717,177,723,307,761,230,383,123,893


In [135]:
categorical_columns = ['Zip','ID','TC']
numerical_columns = ['NRx','PDE','Copay',
                     'Delivered_Doximity','Delivered_Epocrates','Engaged_Doximity',
                     'Engaged_Epocrates','Delivered_Medscape','Engaged_Medscape',
                     'Delivered_Nexgen','Engaged_Nexgen','SpeakerEvent']
hcp_identifier = 'ID'

In [136]:
#Import Page module

dataset_import = ImportData(
    file_path="MMX_Data_PDE_Reduced.csv",
    data_format="long",
    date_format="yyyy-mm",
    date_col='Month',
    categorical_cols=categorical_columns,
    numerical_cols=numerical_columns,
    identifier=hcp_identifier,
    week=False

)

In [137]:
read_data = dataset_import.read_csv()

Here1
Here1


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25620\3061558036.py:77: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_df = data.pivot_table(


In [138]:
read_data.head()

,ID,Copay_Jul19,Copay_Aug19,Copay_Sep19,Copay_Oct19,Copay_Nov19,Copay_Dec19,Copay_Jan20,Copay_Feb20,Copay_Mar20,...,TV_GRPs,Display_Impressions,Display_Clicks,Print_GRPs,TV2_GRPs,Display2_Impressions,Display2_Clicks,Print2_GRPs,TC,Date1
0,14934,4.0,5.0,4.0,2.0,3.0,4.0,5.0,4.0,3.0,...,279,377,80,973,438,622,347,729,C,Jul19
1,14935,5.0,2.0,2.0,1.0,4.0,4.0,1.0,2.0,4.0,...,918,506,723,886,777,943,621,495,C,Jul19
2,14936,3.0,1.0,1.0,3.0,5.0,4.0,2.0,5.0,3.0,...,157,94,998,354,306,650,219,559,C,Jul19
3,14937,4.0,3.0,1.0,2.0,2.0,2.0,4.0,5.0,3.0,...,425,416,978,565,268,375,265,242,T,Jul19
4,14938,2.0,3.0,3.0,1.0,2.0,4.0,2.0,4.0,1.0,...,921,447,242,672,713,276,734,740,C,Jul19


In [139]:
for col in read_data.columns:
    print(col)

ID
Copay_Jul19
Copay_Aug19
Copay_Sep19
Copay_Oct19
Copay_Nov19
Copay_Dec19
Copay_Jan20
Copay_Feb20
Copay_Mar20
Copay_Apr20
Copay_May20
Copay_Jun20
Delivered_Doximity_Jul19
Delivered_Doximity_Aug19
Delivered_Doximity_Sep19
Delivered_Doximity_Oct19
Delivered_Doximity_Nov19
Delivered_Doximity_Dec19
Delivered_Doximity_Jan20
Delivered_Doximity_Feb20
Delivered_Doximity_Mar20
Delivered_Doximity_Apr20
Delivered_Doximity_May20
Delivered_Doximity_Jun20
Delivered_Epocrates_Jul19
Delivered_Epocrates_Aug19
Delivered_Epocrates_Sep19
Delivered_Epocrates_Oct19
Delivered_Epocrates_Nov19
Delivered_Epocrates_Dec19
Delivered_Epocrates_Jan20
Delivered_Epocrates_Feb20
Delivered_Epocrates_Mar20
Delivered_Epocrates_Apr20
Delivered_Epocrates_May20
Delivered_Epocrates_Jun20
Delivered_Medscape_Jul19
Delivered_Medscape_Aug19
Delivered_Medscape_Sep19
Delivered_Medscape_Oct19
Delivered_Medscape_Nov19
Delivered_Medscape_Dec19
Delivered_Medscape_Jan20
Delivered_Medscape_Feb20
Delivered_Medscape_Mar20
Delivered_Medsca

In [146]:
event_identifers = ['SpeakerEvent']


res = {}
avg_table_dict = {}

for event_identifier in event_identifers:
    dist_calc=DistanceCalculation(
    data=read_data,
    num_cols=numerical_columns,
    hcp_identifier=hcp_identifier,
    tc_identifier=tc_identifier,
    event_identifier=event_identifier)

    time_aligned_data=dist_calc.time_align(test_filter_value=test_value,post_cut_off=0)

    data_conversion=dist_calc.column_conversion()

    original_data_conversion = copy.deepcopy(data_conversion)
    dist_calc.data_standardize()

    result=dist_calc.distance_matching(
    control_matches=1,
    agg_matching=True,
    segment_matching=False,
    filter_test_val=test_value,
    filter_control_val=control_value,
    segment_var=segment_var,
    batch_size=batch_size
    )




    # # #Result Page Module

    result_calc=ResultCalculations(
        matching_data=result,
        time_aligned_data=time_aligned_data,
        data_conversion=original_data_conversion,
        hcp_identifier=hcp_identifier
    )
    result_calc.data_merge()
    avgs_data=result_calc.avgs_calculation()
    graph_data_test,graph_data_control=result_calc.graph_data_generation()

    avgs_table,lift=calculate_lift(avgs_data,sales_var)

    # add lift of event_identifier to result dictionary
    res[event_identifier]=lift
    avg_table_dict[event_identifier]=avgs_table

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25620\519834542.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  time_align['Post_Start'] = time_align['Post_Start'].replace(mapping_dict)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25620\519834542.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data['segment_var']="National"


UnboundLocalError: cannot access local variable 'pre_start_var' where it is not associated with a value

In [9]:
# res

{'Speaker_Program': 27.326325078945242, 'Mobile_Alerts': 30.51659942957498}

In [10]:
# avg_table_dict

{'Speaker_Program': {'pre_test_avg': 45.44365,
  'post_test_avg': 54.5375,
  'pre_control_avg': 36.3694,
  'post_control_avg': 34.285},
 'Mobile_Alerts': {'pre_test_avg': 44.8287,
  'post_test_avg': 54.51625,
  'pre_control_avg': 36.382675,
  'post_control_avg': 33.67875}}

In [ ]:
# 
